In [1]:
from google.colab import drive
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
from google.cloud import bigquery

client = bigquery.Client(project='movie-genre-analysis-le-wagon')

query = """
    SELECT *
    FROM `movie-genre-analysis-le-wagon.Secondary_Data.raw_rotten_tomatoes`
"""

df = client.query(query).to_dataframe()

df.head()

,title,url,release_date,critic_score,audience_score
0,The Aristocats,https://www.rottentomatoes.com/m/the_aristocats,1970,NaN,NaN
1,Jenny,https://www.rottentomatoes.com/m/jenny,1970,NaN,NaN
2,End of the Road,https://www.rottentomatoes.com/m/end_of_the_road,2021,NaN,NaN
3,Loving,https://www.rottentomatoes.com/m/loving,1983,NaN,NaN
4,Adventurers,https://www.rottentomatoes.com/m/adventurers,2008,NaN,NaN
...,...,...,...,...,...
12408,The Great Muppet Caper,https://www.rottentomatoes.com/m/great_muppet_...,"Released Jun 26, 1981",0.78,0.78
12409,Unzipped,https://www.rottentomatoes.com/m/unzipped,"Released Aug 11, 1995",0.78,0.78
12410,Valentino: The Last Emperor,https://www.rottentomatoes.com/m/valentino_the...,"Released Mar 18, 2009",0.78,0.78
12411,The Best Exotic Marigold Hotel,https://www.rottentomatoes.com/m/the_best_exot...,"Released May 18, 2012",0.78,0.78


In [ ]:

df.head()

,title,url,release_date,critic_score,audience_score
0,The Aristocats,https://www.rottentomatoes.com/m/the_aristocats,1970,NaN,NaN
1,Jenny,https://www.rottentomatoes.com/m/jenny,1970,NaN,NaN
2,End of the Road,https://www.rottentomatoes.com/m/end_of_the_road,2021,NaN,NaN
3,Loving,https://www.rottentomatoes.com/m/loving,1983,NaN,NaN
4,Adventurers,https://www.rottentomatoes.com/m/adventurers,2008,NaN,NaN


In [ ]:
df.shape

(12413, 5)

In [ ]:
print(df.columns)

Index(['title', 'url', 'release_date', 'critic_score', 'audience_score'], dtype='object')


In [ ]:
print("Dataset Info:")
print(df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12413 entries, 0 to 12412
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           12413 non-null  object 
 1   url             12413 non-null  object 
 2   release_date    12400 non-null  object 
 3   critic_score    9377 non-null   float64
 4   audience_score  10826 non-null  float64
dtypes: float64(2), object(3)
memory usage: 485.0+ KB
None


In [ ]:
print("\nMissing Values:")
print(df.isnull().sum())


Missing Values:
title                0
url                  0
release_date        13
critic_score      3036
audience_score    1587
dtype: int64


In [ ]:
# Convert release_date to datetime format and filter for the last 50 years
df['release_date_clean'] = pd.to_datetime(df['release_date'].str.replace('Released ', ''), errors='coerce')

# Get the current year and calculate the cut-off year (last 50 years)
current_year = dt.datetime.now().year
last_50_years = current_year - 50


In [ ]:
# Filter for movies released in the last 50 years
df_last_50_years = df[df['release_date_clean'].dt.year >= last_50_years]
df_last_50_years.head()

,title,url,release_date,critic_score,audience_score,release_date_clean
2,End of the Road,https://www.rottentomatoes.com/m/end_of_the_road,2021,NaN,NaN,2021-01-01
3,Loving,https://www.rottentomatoes.com/m/loving,1983,NaN,NaN,1983-01-01
4,Adventurers,https://www.rottentomatoes.com/m/adventurers,2008,NaN,NaN,2008-01-01
5,Zig Zag,https://www.rottentomatoes.com/m/zig_zag,1999,NaN,NaN,1999-01-01
7,Equinox,https://www.rottentomatoes.com/m/equinox,2011,NaN,NaN,2011-01-01


In [ ]:
# Drop the 'url' and 'release_date' columns
df_last_50_years.drop(columns=['url', 'release_date'], inplace=True)

# Display the dataset to confirm the columns are dropped
df_last_50_years.head()


,title,critic_score,audience_score,release_date_clean
2,End of the Road,NaN,NaN,2021-01-01
3,Loving,NaN,NaN,1983-01-01
4,Adventurers,NaN,NaN,2008-01-01
5,Zig Zag,NaN,NaN,1999-01-01
7,Equinox,NaN,NaN,2011-01-01


In [ ]:
# Convert critic_score and audience_score to numeric after removing the '%' sign
df_last_50_years['critic_score'] = pd.to_numeric(df_last_50_years['critic_score'].astype(str).str.replace('%', ''), errors='coerce')
df_last_50_years['audience_score'] = pd.to_numeric(df_last_50_years['audience_score'].astype(str).str.replace('%', ''), errors='coerce')

# Scale the scores to a 1-10 range
df_last_50_years['critic_score_scaled'] = df_last_50_years['critic_score'] / 10
df_last_50_years['audience_score_scaled'] = df_last_50_years['audience_score'] / 10

# Display the dataset to confirm the scores are scaled
df_last_50_years[['title', 'critic_score_scaled', 'audience_score_scaled']].head()


,title,critic_score_scaled,audience_score_scaled
2,End of the Road,NaN,NaN
3,Loving,NaN,NaN
4,Adventurers,NaN,NaN
5,Zig Zag,NaN,NaN
7,Equinox,NaN,NaN


In [ ]:
# Calculate the number and percentage of null values in the scaled scores
null_critic_score_last_50 = df_last_50_years['critic_score_scaled'].isnull().sum()
null_audience_score_last_50 = df_last_50_years['audience_score_scaled'].isnull().sum()

In [ ]:
# Total number of rows in the last 50 years dataset
total_rows_last_50 = len(df_last_50_years)


In [ ]:
# Calculate the percentage of nulls
percent_null_critic_last_50 = (null_critic_score_last_50 / total_rows_last_50) * 100
percent_null_audience_last_50 = (null_audience_score_last_50 / total_rows_last_50) * 100

print(f"Number of null values in critic_score_scaled: {null_critic_score_last_50}")
print(f"Percentage of null values in critic_score_scaled: {percent_null_critic_last_50:.2f}%")

print(f"Number of null values in audience_score_scaled: {null_audience_score_last_50}")
print(f"Percentage of null values in audience_score_scaled: {percent_null_audience_last_50:.2f}%")

Number of null values in critic_score_scaled: 2067
Percentage of null values in critic_score_scaled: 54.04%
Number of null values in audience_score_scaled: 1188
Percentage of null values in audience_score_scaled: 31.06%


In [ ]:
# Drop rows where either critic_score_scaled or audience_score_scaled is null
df_last_50_years_cleaned = df_last_50_years.dropna(subset=['critic_score_scaled', 'audience_score_scaled'])

# Display the cleaned dataset
df_last_50_years_cleaned.head()

# Check the number of rows after dropping the nulls
print(f"Number of rows after dropping nulls: {len(df_last_50_years_cleaned)}")


Number of rows after dropping nulls: 1722


In [ ]:
# Drop the non-scaled columns 'critic_score' and 'audience_score'
df_last_50_years_cleaned.drop(columns=['critic_score', 'audience_score'], inplace=True)

# View the first few rows to confirm the columns are dropped
df_last_50_years_cleaned.head()

,title,release_date_clean,critic_score_scaled,audience_score_scaled
126,Breach,2007-02-16,8.4,6.7
129,The Deep End,2001-08-01,8.2,5.9
266,The Beguiled,2017-06-30,7.8,4.8
279,Willard,2003-03-14,6.4,5.1
294,Shaft,2019-06-14,3.4,9.4


In [ ]:
df_last_50_years_cleaned.to_csv('cleaned_rotten_tomatoes_last_50_years.csv', index=False)

In [ ]:
# Count the number of unique titles
unique_title_count = df_last_50_years_cleaned['title'].nunique()

# Print the result
print(f"Number of unique titles: {unique_title_count}")


Number of unique titles: 6061
